# 2018 Later 5 Years

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2025-01-13  


## 1. Import packages

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
sys.path.insert(1,'../../../')
from config import conf as cfg
import rxnrecer as production
from modules import commonfunction as cmfunc
from tqdm import tqdm
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
import json
import numpy as np
import hashlib
from collections import defaultdict
from tools import btools
from evaluation import evTools
from modules.embedding import seqEmbedding as ebdseq
from modules.embedding import t5Embedding as ebdt5
import pandas as pd
from methods import CaclSimiProteinWithfoldNum as calsimi
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn3

FIRST_TIME_RUN = True # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. TestSet for Newly 5 Years

In [58]:
test_set = pd.read_feather(cfg.FILE_DS_TEST)
# test_set = test_set[['uniprot_id','seq', 'reaction_id', 'ec_number', 'label']].rename(columns={'reaction_id': 'rxn_groundtruth', 'ec_number': 'ec_groundtruth', 'label': 'rxn_groundtruth_label'})
test_set.head(3)

train_set = pd.read_feather(cfg.FILE_DS_TRAIN)

In [22]:
featureBank = pd.read_feather(cfg.FILE_PRODUCTION_FEATURES)
dict_featureBank = pd.Series( featureBank['uniprot_id'],featureBank.index.values).to_dict()
with open(cfg.DICT_UNIPROT_RHEA, "r") as json_file:
    dict_uniprot2rhea = json.load(json_file)
embd_esm = pd.read_feather(f'../../temp/5874051d86b8daacace8553da344cf88_esm.feather')

In [59]:
esm_cos = production.get_top_protein_simi(x_feature=np.vstack(featureBank.esm), 
                                y_feature=np.vstack(embd_esm.esm), 
                                y_uniprot_id=embd_esm.id, 
                                dict_featureBank=dict_featureBank, 
                                dict_uniprot2rhea = dict_uniprot2rhea,
                                topk=10).rename(columns={'simi':'esm'})


In [45]:
esm_cos['esm'] = esm_cos['esm'].apply(lambda x: [item for item in x])

In [60]:
esm_cos.to_pickle('../../temp/temp1.pkl')
    

In [61]:
pd.read_pickle('../../temp/temp1.pkl')

,uniprot_id,esm
0,C0HKD2,"[(-, 0.99126), (-, 0.990677), (-, 0.989989), (..."
1,C0HKD3,"[(-, 0.99126), (-, 0.990677), (-, 0.989989), (..."
2,A7B3K3,[(RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;...
3,C0HK72,"[(-, 1.0), (-, 0.996938), (-, 0.996778), (-, 0..."


In [49]:
esm_cos

,uniprot_id,esm
0,C0HKD2,"[(-, 0.99126), (-, 0.990677), (-, 0.989989), (..."
1,C0HKD3,"[(-, 0.99126), (-, 0.990677), (-, 0.989989), (..."
2,A7B3K3,[(RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;...
3,C0HK72,"[(-, 1.0), (-, 0.996938), (-, 0.996778), (-, 0..."


In [53]:
esm_cos.to_hdf(f'{cfg.TEMP_DIR}aaaa_simi_t5.h5', key='data', mode='w')

/tmp/ipykernel_2530538/2492989342.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['uniprot_id', 'esm'], dtype='object')]

  esm_cos.to_hdf(f'{cfg.TEMP_DIR}aaaa_simi_t5.h5', key='data', mode='w')


In [64]:
input_protein_df = pd.read_feather(cfg.FILE_DS_TEST)[['uniprot_id', 'seq']]
res  = production.step_by_step_prediction(input_protein_df.head(35), cfg.FILE_DS_DICT_RXN2ID, getEquation=False, Ensemble=True, batch_size=100)
res

Step 1: Preparing input data
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 35 proteins


  0%|          | 0/4 [00:00<?, ?it/s]/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting reactions: 100%|██████████| 5/5 [00:00<00:00,  5.26it/s]


Transferred model to GPU


100%|██████████| 1/1 [00:03<00:00,  3.94s/it]
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 25%|██▌       | 1/4 [00:23<01:11, 23.94s/it]/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multith

Transferred model to GPU


...

100%|██████████| 1/1 [00:06<00:00,  6.62s/it]
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 75%|███████▌  | 3/4 [01:11<00:23, 23.96s/it]/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multith

Transferred model to GPU


100%|██████████| 1/1 [00:04<00:00,  4.22s/it]
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 4/4 [01:33<00:00, 23.27s/it]

Step 5: Saving results to /hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/dict/dict_rxn2id.json (format=tsv)


,uniprot_id,esm,unirep,t5,RXNRECer,RXNRECer_with_prob,ensemble
0,A9JLI2,"[(-, 0.99989)]","[(-, 0.997023)]","[(-, 0.999031)]",-,{'-': 0.999997},{'-': 0.999997}
1,A9JLI3,"[(-, 0.999648)]","[(-, 0.998804)]","[(-, 0.991779)]",-,{'-': 0.999998},{'-': 0.999998}
2,A9JLI5,"[(-, 1.0)]","[(-, 1.0)]","[(-, 1.0)]",-,{'-': 0.999996},{'-': 1.0}
3,A9JLI7,"[(-, 0.999283)]","[(-, 0.993645)]","[(-, 0.995576)]",-,{'-': 0.999997},{'-': 0.999997}
4,B5KVH4,"[(-, 0.995613)]","[(-, 0.962547)]","[(-, 0.993225)]",-,{'-': 0.999998},{'-': 0.999998}
5,C0HLM8,"[(-, 0.990871)]","[(-, 0.950686)]","[(RHEA:15801, 0.940819)]",-,{'-': 0.999989},"{'-': 0.999989, 'RHEA:15801': 0.940819}"
6,C0HLR7,"[(-, 0.983171)]","[(-, 0.929157)]","[(-, 0.893383)]",-,{'-': 0.999994},{'-': 0.999994}
7,C0HLM9,"[(-, 0.991106)]","[(-, 0.94877)]","[(-, 0.944557)]",-,{'-': 0.999941},{'-': 0.999941}
8,A9NJG2,"[(-, 0.999305)]","[(-, 0.994395)]","[(-, 0.997292)]",-,{'-': 0.999997},{'-': 0.999997}
9,C0H4V6,"[(-, 0.997035)]","[(-, 0.977663)]","[(-, 0.995675)]",-,{'-': 0.999996},{'-': 0.999996}


In [71]:
from transformers import T5Tokenizer, T5EncoderModel
tokenizer = T5Tokenizer.from_pretrained('/hpcfs/fhome/shizhenkun/.cache/huggingface/hub/models--Rostlab--ProstT5/snapshots/d7d097d5bf9a993ab8f68488b4681d6ca70db9e5', do_lower_case=False)


In [ ]:
model_dir=

In [70]:
model = T5EncoderModel.from_pretrained("Rostlab/ProstT5")

In [ ]:
input_protein_df = pd.read_feather(cfg.FILE_DS_TEST)[['uniprot_id', 'seq']]
res  = production.step_by_step_prediction(input_protein_df.iloc[0:100], cfg.FILE_DS_DICT_RXN2ID, getEquation=False, Ensemble=True)
res

In [17]:
res

,uniprot_id,esm,unirep,t5,input_id,RXNRECer,RXNRECer_with_prob
0,F1P963,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,F1P963,RHEA:67392;RHEA:67604;RHEA:65396;RHEA:67608;RH...,"{'RHEA:67392': 0.983453, 'RHEA:67604': 0.98466..."
1,Q7ZWC3,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,[(RHEA:31583;RHEA:67392;RHEA:31575;RHEA:65396;...,Q7ZWC3,RHEA:67392;RHEA:67604;RHEA:65396;RHEA:67608;RH...,"{'RHEA:67392': 0.967587, 'RHEA:67604': 0.97274..."
2,A0A517FNC7,"[(RHEA:54832;RHEA:54808;RHEA:55164, 0.984114)]","[(RHEA:54832, 0.952145)]","[(RHEA:54832, 0.990778)]",A0A517FNC7,RHEA:66620,{'RHEA:66620': 0.088376}
3,A0A517FND0,"[(RHEA:55056, 0.988275)]","[(RHEA:35499, 0.918117)]","[(RHEA:55056, 0.986594)]",A0A517FND0,RHEA:43728,{'RHEA:43728': 0.036686}
4,P0DUU4,"[(-, 0.985649)]","[(-, 0.956612)]","[(-, 0.942892)]",P0DUU4,-,{'-': 0.999993}
5,Q572D3,"[(-, 0.989133)]","[(-, 0.959387)]","[(-, 0.897574)]",Q572D3,-,{'-': 0.999996}
6,D0NPN8,"[(-, 0.989133)]","[(-, 0.959387)]","[(-, 0.897574)]",D0NPN8,-,{'-': 0.999996}
7,E2DWQ7,"[(-, 0.989185)]","[(-, 0.957658)]","[(-, 0.901532)]",E2DWQ7,-,{'-': 0.999996}
8,P0DOO9,"[(None, 1.0)]","[(None, 1.0)]","[(None, 1.0)]",P0DOO9,-,{'-': 0.999997}
9,P0DOP0,"[(None, 1.0)]","[(None, 1.0)]","[(None, 1.0)]",P0DOP0,-,{'-': 0.999997}


## 3.Bagging

## 2 加载结果文件

## 4. Add gene data

## 5. 增加各种标签

## 6. Prepare Metabolic Network data
### 6.1 Metabolites